In [1]:
import tensorflow as tf
import numpy as np

import sgld_tf

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
directory = '/tmp/tf/distrib_sgld/'
name = 'model.ckpt-101'

In [4]:
cnn = sgld_tf.CNN(train_data, train_labels,
                  eval_data, eval_labels,
                  bs=512, model_dir=directory,
                 optimize_method="sgld",
                 learning_rate=0.001,
                 save_checkpoint_steps=10,
                 keep_checkpoint_max=1000000, 
                 shuffle=True)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf/distrib_sgld/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 1000000, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb26298f898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
predictions = cnn.estimator.predict(input_fn = cnn.eval_input_fn,
                                    checkpoint_path=directory + name)

In [6]:
next(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf/distrib_sgld/model.ckpt-101
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'classes': 9,
 'probabilities': array([ 0.09771957,  0.08552644,  0.10373189,  0.08976313,  0.10429506,
         0.09510571,  0.10200264,  0.10704725,  0.10416567,  0.1106426 ], dtype=float32)}

In [5]:
def aggregate_predictions(root,
                          names,
                         train_data, 
                         train_labels,
                          eval_data,
                          eval_labels,
                         method):
    cnn = sgld_tf.CNN(train_data, train_labels,
                      eval_data, eval_labels,
                      bs=512, model_dir=root,
                     optimize_method=method,
                     learning_rate=0.001,
                     save_checkpoint_steps=10,
                     keep_checkpoint_max=1000000, 
                     shuffle=True)
    all_models_predictions = []
    for name in names:
        single_model_predictions = []
        i = 0
        predictions = cnn.estimator.predict(input_fn = cnn.eval_input_fn,
                                           checkpoint_path=root + name,
                                           yield_single_examples=False)
        n = eval_labels.shape[0]
        label_batches = (eval_labels[i:i + cnn.bs] for i in np.arange(0, n, cnn.bs))
        for labels, prediction in zip(label_batches, predictions):
            single_model_predictions.append(prediction['probabilities'][:len(labels)])
            
        all_models_predictions.append(np.vstack(single_model_predictions))
        
    summed_probabilities = np.sum(np.array(all_models_predictions), axis=0)
    predicted_labels = np.argmax(summed_probabilities, axis=1)
    accuracy = np.mean(predicted_labels == eval_labels)
    return accuracy

In [14]:
agg_pred = aggregate_predictions(directory, ['model.ckpt-101',
                                             'model.ckpt-101',
                                             'model.ckpt-101',
                                             'model.ckpt-101',
                                             'model.ckpt-101'], train_data,
                                train_labels,
                                eval_data,
                                eval_labels,
                                "sgld")

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf/distrib_sgld/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 1000000, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb25b9f46d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf/distrib_sgld/model.ckpt-101
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflo

In [15]:
agg_pred

0.1038

In [10]:
import os
os.listdir(directory)

['model.ckpt-1882.data-00000-of-00001',
 'model.ckpt-7789.index',
 'model.ckpt-5783.index',
 'model.ckpt-381.meta',
 'model.ckpt-7489.index',
 'model.ckpt-3883.index',
 'model.ckpt-4552.data-00000-of-00001',
 'model.ckpt-8500.index',
 'model.ckpt-4732.index',
 'model.ckpt-1101.meta',
 'model.ckpt-6113.meta',
 'model.ckpt-4482.index',
 'model.ckpt-5483.index',
 'model.ckpt-6723.data-00000-of-00001',
 'model.ckpt-5553.meta',
 'model.ckpt-1922.meta',
 'model.ckpt-3143.index',
 'model.ckpt-841.meta',
 'model.ckpt-2453.meta',
 'model.ckpt-4982.meta',
 'model.ckpt-8470.index',
 'model.ckpt-3163.meta',
 'model.ckpt-6083.data-00000-of-00001',
 'model.ckpt-6303.data-00000-of-00001',
 'model.ckpt-7749.index',
 'model.ckpt-4912.index',
 'model.ckpt-991.data-00000-of-00001',
 'model.ckpt-5212.data-00000-of-00001',
 'model.ckpt-8300.index',
 'model.ckpt-9501.data-00000-of-00001',
 'model.ckpt-8610.meta',
 'model.ckpt-8560.data-00000-of-00001',
 'model.ckpt-8880.data-00000-of-00001',
 'model.ckpt-76

In [48]:
[(l.shape, i) for (i, l) in enumerate(label_batches)]

[((512,), 0),
 ((512,), 1),
 ((512,), 2),
 ((512,), 3),
 ((512,), 4),
 ((512,), 5),
 ((512,), 6),
 ((512,), 7),
 ((512,), 8),
 ((512,), 9),
 ((512,), 10),
 ((512,), 11),
 ((512,), 12),
 ((512,), 13),
 ((512,), 14),
 ((512,), 15),
 ((512,), 16),
 ((512,), 17),
 ((512,), 18),
 ((272,), 19)]

In [49]:
[(agg.shape, i) for (i, agg) in enumerate(agg_pred)]

[((512, 10), 0),
 ((512, 10), 1),
 ((512, 10), 2),
 ((512, 10), 3),
 ((512, 10), 4),
 ((512, 10), 5),
 ((512, 10), 6),
 ((512, 10), 7),
 ((512, 10), 8),
 ((512, 10), 9),
 ((512, 10), 10),
 ((512, 10), 11),
 ((512, 10), 12),
 ((512, 10), 13),
 ((512, 10), 14),
 ((512, 10), 15),
 ((512, 10), 16),
 ((512, 10), 17),
 ((512, 10), 18),
 ((512, 10), 19)]

In [39]:
lab.shape

(512,)